<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_realize/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [0]:
%tensorflow_version 1.x

In [0]:
!pip install keras_bert

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [0]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

In [0]:
!git checkout equal_realize
!git pull

# Mount GDriver
Perpare Colab environment.

In [0]:
from driver_amount import addh

# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess
import config

In [0]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [0]:
import json
with open(addh + config.TAG_VOCAB_PATH, "w") as fd:
    json.dump(tag_vocab, fd)

In [0]:
tag_vocab

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [0]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)

In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [0]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

In [0]:
bert_model.summary()

In [0]:
model.summary()

# Fine-tuning

In [0]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)

In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
model.fit(
    x=[train_token_id_seqs, train_segment_seqs],
    y=[train_tag_id_seqs], 
    batch_size=4, 
    epochs=15,
    verbose=1,
    validation_data=[[test_token_id_seqs, test_segment_seqs], test_tag_id_seqs],
    callbacks=[epoch_callback]
)

In [0]:
model.save(addh + config.MODEL_PATH)